# Modification du CSV :
**On ajoute une colonne dans le csv** tel quel `geo_shape` n'est pas exploitable.
On ajoute une colonne `forme_geo` qui est une interpretation de `geo_shape`.

In [1]:
import json
import pandas as pd
import numpy as np

Lecture du .csv

In [2]:
chantier = pd.read_csv('./csv_file/chantiers-a-paris.csv', delimiter=';')

In [3]:
chantier.head()

,Référence Chantier,Code postal arrondissement - Commune,Date début du chantier,Date fin du chantier,Responsable du chantier,Maîtrise d'ouvrage principale,Surface (m2),Synthèse - Nature du chantier,Encombrement espace public,Impact stationnement,Identifiant demande CITE,Identifiant Chantier CITE,geo_shape,geo_point_2d
0,EC262569,75008,2022-02-28,2024-04-30,Tiers (travaux sur bâtiment),Propriétés - Co-propriétés publiques et privées,144.42,Construction ou réhabilitation d'immeuble,EMPRISE_TROTTOIR,NaN,534314,172512,"{""coordinates"": [[[2.3002816159215382, 48.8717...","48.87171473163943, 2.300472796020154"
1,EC267934,75015,2022-06-06,2023-09-30,Opérateurs de réseau (gaz-électricité-RATP-etc),CPCU,94.88,Travaux sur réseau de chauffage urbain,"EMPRISE_CHAUSSEE,EMPRISE_TROTTOIR",NaN,577778,178217,"{""coordinates"": [[[2.292899247699561, 48.85604...","48.85610765531542, 2.2929003924589715"
2,EC268310,75006,2022-04-01,2023-05-15,Tiers (travaux sur bâtiment),Propriétés - Co-propriétés publiques et privées,164.77,Construction ou réhabilitation d'immeuble,"EMPRISE_CHAUSSEE,EMPRISE_TROTTOIR","STATIONNEMENT_PAYANT,ZONE_LIVRAISON",569382,176946,"{""coordinates"": [[[2.32580213112853, 48.847082...","48.846983726393866, 2.3259329706698515"
3,EC268626,NaN,2022-02-14,2023-06-16,Tiers (travaux sur bâtiment),Propriétés - Co-propriétés publiques et privées,0.96,Construction ou réhabilitation d'immeuble,EMPRISE_CHAUSSEE,NaN,579401,178537,"{""coordinates"": [[[2.384292595886623, 48.82593...","48.825931811335515, 2.384302128015279"
4,EC286541,75005,2022-12-19,2023-05-12,Tiers (travaux sur bâtiment),Propriétés - Co-propriétés publiques et privées,7.18,Construction ou réhabilitation d'immeuble,EMPRISE_TROTTOIR,NaN,655170,189925,"{""coordinates"": [[[2.344365865727618, 48.84455...","48.84454136424176, 2.3444310393485357"


### Extraction de la donnée :
On cherche ici obtenir la forme géométrique du chantier.

In [4]:
def to_list(df):
	list_type = []
	for i in range(6982):
		try :
			geo = df[i]
			geo = json.loads(geo)   # utilisation de json pour lire la string comme un dict
			list_type.append(geo)
		except Exception as e:
			list_type.append(np.NaN)    # en cas d'exception on ajoute nan
			print(e)

	return list_type

In [6]:
def get_shape(list):
	polygones = {3:"triangle", 4:"quadrilatère", 5:"pentagone", 6:"hexagone", 7:"heptagone", 8:"octogone", 9:"nonagone", 10:"décagone", 11:"hendécagone", 12:"dodécagone"}

	nb_err = 0

	for i, geo in enumerate(list):
		try :
			nb_cotes = len(geo['coordinates'][0])
			if nb_cotes in polygones.keys():    # cherche parmi les clés
				list[i] = polygones[nb_cotes]
			elif geo == np.NaN:                 # si c'est deja nan car to_list ne peut pas convertir
				nb_err +=1
				continue
			else :                              # si plus de 12 cotés
				list[i] = 'autre'
		except Exception as e:                  # si on catch une exception
			list[i] = np.NaN
			print(e)
			nb_err +=1

	print(f"le nombre d'err est {nb_err}")
	return list

***

In [7]:
list_geo_shape = to_list(chantier['geo_shape'])
new_column = get_shape(list_geo_shape)
chantier['forme_geo'] = new_column

the JSON object must be str, bytes or bytearray, not float
the JSON object must be str, bytes or bytearray, not float
the JSON object must be str, bytes or bytearray, not float
the JSON object must be str, bytes or bytearray, not float
the JSON object must be str, bytes or bytearray, not float
the JSON object must be str, bytes or bytearray, not float
the JSON object must be str, bytes or bytearray, not float
the JSON object must be str, bytes or bytearray, not float
the JSON object must be str, bytes or bytearray, not float
the JSON object must be str, bytes or bytearray, not float
the JSON object must be str, bytes or bytearray, not float
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subscriptable
'float' object is not subsc

### Analyse des erreurs et des distributions des valeurs du dataset

In [10]:
chantier['forme_geo'].value_counts()

pentagone       4894
heptagone        523
autre            433
hexagone         347
nonagone         266
octogone         189
décagone         106
hendécagone      104
dodécagone        69
quadrilatère      40
Name: forme_geo, dtype: int64

In [15]:
chantier['forme_geo'].describe()

count          6971
unique           10
top       pentagone
freq           4894
Name: forme_geo, dtype: object

***
### Export du dataset en .csv :
Le fichier va être exporté dans le même dossier que `chantier.ipybn`

In [16]:
from pathlib import Path
filepath = Path('chantier_exp.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
chantier.to_csv(filepath)